In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy.io import wavfile
import re
import seaborn as sns
import random
from sklearn import svm
import librosa
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.utils.np_utils import to_categorical


In [22]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [23]:
# test data and test data directory 

data_dir = "C:/Users/gaozh/Downloads/ELEC378_FinalProject/data/data"

test_dir = "C:/Users/gaozh/Downloads/ELEC378_FinalProject/test/test"



Containis function for feature extraction. Takes a directory and a boolean indicating wheather to split the data or not. For cross validation use the data directory (labeled data) and split = true. Run all the way to the cell that prints out accuracy score. For testing model on the unlabled data in test folder (those that are named sample001, etc), run the cells below the accuracy score cell will give you a .csv file ready for submission on kaggle. 

In [ ]:
#@title Feature Extraction func
def FeatureExtraction (dir, split=True): 
 
  '''
  Function for feature extraction. Change this section to change what features we are using. 

  input: a directory of the data. Will split to test and train, a boolean: if split is true, will extract features and labels and store them in 4 arrays:
  train_data, train_label, test_data, test_label. if split is false, function will extract all data provided in the directory and output 2 arrays with values 2 arrays that are empty. 
  Use labeled data and split = false will give you  (train_data, train_label); use unlabled data and split = false will give you (test_data, test_label).

  output: data matrix and lables for both test and train. 

  '''
  audio_files = [f for f in os.listdir(dir) if f.endswith(".wav")]
  
  train_files, test_files = train_test_split(audio_files, test_size=0.2)

  train_data = []
  train_label= []
  test_data = []
  test_label = []  
  
  
  for file_name in audio_files:

      # Load audio
      file_path = os.path.join(dir, file_name)
      raw_audio, sr = librosa.load(file_path)

      # if audio is too short, append zeros after it. 
      if np.shape(raw_audio)[0] < 80000: 
          padded_audio = np.pad(raw_audio, [(0, 80000 - np.shape(raw_audio)[0])], mode='constant')
      else:
          padded_audio = raw_audio

      # now slice so that we guarantee that each array has the same length
      audio = padded_audio[20000:80000]

      # feature extraction

      mfccs = librosa.feature.mfcc(y=audio, sr = 22050, n_mfcc = 20)
      delta_mfccs = librosa.feature.delta(mfccs)
      delta2_mfccs = librosa.feature.delta(mfccs, order=2)

      #chroma_stft = librosa.feature.chroma_stft(y = audio, sr=22050)
      mel_spec = librosa.feature.melspectrogram(y = audio, sr = 22050, n_mels = 60)
      #gfccs = librosa.feature.gfcc(y, sr=sr, n_mfcc=20)

      # Concatenate
      features = np.concatenate([mfccs, delta_mfccs, delta2_mfccs, mel_spec], axis = 0)
      #features = mfccs

      label_map = {'angry': 0, 'calm': 1, 'disgust': 2, 'fearful': 3, 'happy': 4, 'neutral': 5, 'sad': 6, 'surprised': 7}
      label = file_name.split(".")[0]

      #unlabeled data
      if label[:-3] == "sample":
        test_data.append(features)
        test_label.append(label)

      #labeled data
      else:
        if split: 
          if file_name in train_files:
            train_data.append(features)
            label = re.sub('[^a-z]', "", label)
            train_label.append(label_map[label])

          elif file_name in test_files: 
            test_data.append(features)
            label = re.sub('[^a-z]', "", label)
            test_label.append(label_map[label])

        elif not split: 
          
          train_data.append(features)
          label = re.sub('[^a-z]', "", label)
          train_label.append(label_map[label])
  

  train_data = np.array(train_data)
  train_label = np.array(train_label)
  test_data = np.array(test_data)
  test_label = np.array(test_label) 

  print("mfccs size is:", np.shape(mfccs))
  print("mel spectrogram is", np.shape(mel_spec))
  return train_data, train_label, test_data, test_label



In [15]:
X_train, y_train, X_test, y_test = FeatureExtraction(data_dir, split = True)
y_train_ohe = to_categorical(y_train)
y_test_ohe = to_categorical(y_test)


mfccs size is: (20, 118)
mel spectrogram is (60, 118)


In [ ]:
#@title SVM
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(kernel = "linear"))
clf.fit(X_train, y_train)

y_predicted= clf.predict(X_test)

accuracy = accuracy_score(y_test, y_predicted)

print(accuracy)


In [16]:
import keras
from keras.layers import Conv1D, Embedding, Dropout, MaxPooling1D, GlobalMaxPooling1D, Dense
from keras.optimizers import Adam
from keras import Model
from keras.callbacks import EarlyStopping

In [17]:
num_classes = 8
def build_cnn(input_shape, num_classes):
    model = tf.keras.Sequential()
    
    # Convolutional layers
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='valid', input_shape=input_shape))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(128, (3, 3), padding='valid'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding='valid'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.1))

    # Dense layers
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Dense(num_classes))
    model.add(tf.keras.layers.Activation('softmax'))
    
    
    return model

In [18]:
#num frames is number of wav files
es = EarlyStopping(monitor='val_loss',
                   mode='min',
                   restore_best_weights=True,
                   patience=10,
                   verbose=1)

#num frames is number of wav files
model = build_cnn((X_train.shape[1], X_train.shape[2], 1), num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train_ohe, batch_size=32, epochs=30, validation_data=(X_test, y_test_ohe), callbacks=[es])


# plot
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training and validation error')
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

Epoch 1/30
29/29 [==============================] - 38s 1s/step - loss: 1.7830 - accuracy: 0.3867 - val_loss: 18.4263 - val_accuracy: 0.1511
Epoch 2/30
29/29 [==============================] - 34s 1s/step - loss: 1.3225 - accuracy: 0.5489 - val_loss: 7.5130 - val_accuracy: 0.1422
Epoch 3/30
29/29 [==============================] - 35s 1s/step - loss: 1.0075 - accuracy: 0.6456 - val_loss: 3.0496 - val_accuracy: 0.3511
Epoch 4/30
 4/29 [===>..........................] - ETA: 29s - loss: 0.7222 - accuracy: 0.7656

KeyboardInterrupt: 

In [ ]:
# extract all labeled data
X_train, y_train, dummy1, dummy2 = FeatureExtraction(data_dir, split = False)
# extract all unlabeled data
dummy1, dummy2, X_test, y_test = FeatureExtraction(test_dir, split = False)
y_train_ohe = to_categorical(y_train)


In [ ]:
# this cell is to export a .csv file to submit (filename & predicted label) 
model = build_cnn((X_train.shape[1], X_train.shape[2], 1), num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.fit(X_train, y_train_ohe, batch_size=32, epochs=25)
y_predicted = model.predict(X_test)



In [ ]:

y_predicted_ohv = np.argmax(y_predicted, axis = 1)


In [ ]:
print(y_predicted_ohv)
label_map = {'angry': 0, 'calm': 1, 'disgust': 2, 'fearful': 3, 'happy': 4, 'neutral': 5, 'sad': 6, 'surprised': 7}

def dict_search(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key

y_predicted_label = []
for i in y_predicted_ohv: 
    val = dict_search(label_map, i)
    y_predicted_label.append(val)


print(y_predicted_label)

In [ ]:
import csv

def make_csv(unlabeled_file_name, y_predicted_label):
  with open('prediction.csv', mode='w', newline='') as file:
      writer = csv.writer(file)
      writer.writerow(['filename', 'label'])
      for i in range(len(unlabeled_file_name)):
        writer.writerow([unlabeled_file_name[i], y_predicted_label[i]])
  file.close()

make_csv(y_test, y_predicted_label)